In [1]:
# read data
# build T-learner 
# impute values using T-learner as ground truth 
# run bootstrap ci

### End goal: compare standard CI vs bca CI

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng
import random
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv('GerberGreenLarimer_APSR_2008_social_pressure.csv')
df = df[df['treatment'].isin([' Control',' Neighbors'])]

In [4]:
# df = pd.read_csv('GerberGreenLarimer_APSR_2008_social_pressure.csv')
df['treatment'] = np.where(df.treatment == ' Control',0,1)
df['voted'] = np.where(df.voted == 'Yes', 1, 0)
df['sex'] = np.where(df.sex == 'male',1, 0)
df['g2000'] = np.where(df.g2000 == 'yes', 1, 0)
df['g2002'] = np.where(df.g2002 == 'yes', 1, 0)
df['g2004'] = np.where(df.g2004 == 'yes', 1, 0)
df['p2000'] = np.where(df.p2000 == 'yes', 1, 0)
df['p2002'] = np.where(df.p2002 == 'yes', 1, 0)
df['p2004'] = np.where(df.p2004 == 'Yes', 1, 0)

In [5]:
cts_variables_names = ["yob","treatment","cluster","hh_id","hh_size","numberofnames","p2004_mean","g2004_mean"]
binary_variables_names = ["sex","g2000", "g2002", "p2000", "p2002", "p2004"]
# for column in binary_variables_names:
#     if column == 'sex':
#         df[column] = np.where(df[column] == ' male',1,0)
#     else:
#         df[column] = df[column].str.lower()
#         df[column] = np.where(df[column] == ' yes',1,0)
scaled_cts_covariates = StandardScaler().fit_transform(df[cts_variables_names])
binary_covariates = df[binary_variables_names]
d = pd.DataFrame(np.concatenate((scaled_cts_covariates, binary_covariates), axis=1), 
                        columns=cts_variables_names+binary_variables_names, index=df.index)
d["W"] = df["treatment"]
d["Y"] = df["voted"]


In [6]:
# df = pd.read_csv('sp.csv.xz')
# cts_variables_names = ["yob", "hh_size", "totalpopulation_estimate",
#                          "percent_male", "median_age",
#                          "percent_62yearsandover",
#                          "percent_white", "percent_black",
#                          "percent_asian", "median_income",
#                          "employ_20to64", "highschool", "bach_orhigher",
#                          "percent_hispanicorlatino"]
# binary_variables_names = ["sex","g2000", "g2002", "p2000", "p2002", "p2004"]
# scaled_cts_covariates = StandardScaler().fit_transform(df[cts_variables_names])
# binary_covariates = df[binary_variables_names]
# d = pd.DataFrame(np.concatenate((scaled_cts_covariates, binary_covariates), axis=1), 
#                         columns=cts_variables_names+binary_variables_names, index=df.index)
# d["W"] = df["treat_neighbors"]
# d["Y"] = df["outcome_voted"]

In [7]:
# T learner
def T_learner(d):
    # w == 0
    d0 = d[d['W'] == 0]
    X_d0 = d0.drop(columns=['W','Y'])
    y_d0 = d0['Y']
    clf_d0 = RandomForestClassifier()
    clf_d0.fit(X_d0,y_d0)

    # w == 1
    d1 = d[d['W'] == 1]
    X_d1 = d1.drop(columns=['W','Y'])
    y_d1 = d1['Y']
    clf_d1 = RandomForestClassifier()
    clf_d1.fit(X_d1,y_d1)

    pred_y0 = clf_d0.predict_proba(d.drop(columns= ['W','Y']))[:,1]
    pred_y1 = clf_d1.predict_proba(d.drop(columns= ['W','Y']))[:,1]
    
    ate_score = (sum(pred_y1) - sum(pred_y0)) / len(pred_y0)
    return ate_score, clf_d0, clf_d1

In [10]:
from metalearners import * 

In [11]:
t_learner = Tlearner(RandomForestClassifier(),RandomForestClassifier())
t_learner.fit(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [12]:
ite, yhat_ts, yhat_cs, rmse = t_learner.get_ite(X = d.drop(columns=['W','Y']), treatment = d['W'], y =  d['Y'])

In [13]:
np.mean(ite)

0.08207330339346212

In [14]:
_, clf_d0, clf_d1 = T_learner(d)
pred_y0 = clf_d0.predict(d.drop(columns= ['W','Y']))
pred_y1 = clf_d1.predict(d.drop(columns= ['W','Y']))

In [15]:
new_y0 = []
new_y1 = []
for i in range(len(d)):
    if d.iloc[i].W == 0.0:
        new_y0.append(d.iloc[i].Y)
        new_y1.append(pred_y1[i])
    else:
        new_y0.append(pred_y0[i])
        new_y1.append(d.iloc[i].Y)

In [16]:
d['new_y0'] = new_y0
d['new_y1'] = new_y1

In [17]:
d

,yob,treatment,cluster,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,sex,g2000,g2002,p2000,p2002,p2004,W,Y,new_y0,new_y1
5,1.716448,-0.446935,-1.727258,-1.727383,1.031783,0.139749,-1.443668,-0.259034,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
6,0.195026,-0.446935,-1.727258,-1.727383,1.031783,0.139749,-1.443668,-0.259034,0.0,1.0,1.0,0.0,1.0,0.0,0,1,1.0,0.0
7,-0.012441,-0.446935,-1.727258,-1.727383,1.031783,0.139749,-1.443668,-0.259034,1.0,1.0,1.0,0.0,1.0,0.0,0,1,1.0,0.0
8,0.817426,-0.446935,-1.727258,-1.727364,-0.233199,0.139749,-1.443668,-0.259034,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0.0,0.0
9,0.748270,-0.446935,-1.727258,-1.727364,-0.233199,0.139749,-1.443668,-0.259034,1.0,1.0,1.0,0.0,1.0,0.0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344079,-0.911463,-0.446935,1.736207,1.736332,-0.233199,0.139749,0.952474,0.392520,1.0,1.0,1.0,0.0,1.0,1.0,0,1,1.0,1.0
344080,-0.081596,-0.446935,1.736207,1.736351,-0.233199,0.139749,-0.378716,-0.910586,1.0,1.0,1.0,1.0,0.0,1.0,0,0,0.0,0.0
344081,0.125870,-0.446935,1.736207,1.736351,-0.233199,0.139749,-0.378716,-0.910586,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
344082,-1.326396,-0.446935,1.736207,1.736371,-0.233199,0.139749,0.153760,-0.910586,1.0,1.0,1.0,1.0,1.0,1.0,0,1,1.0,1.0


In [18]:
full_ate=0.08

In [19]:
def get_bootstrap_stats(boot_estimates, full_estimates, alpha=0.05):
    
    est_stat = []
    signif_level = -norm.ppf(alpha/2)
    est_boot = np.array(boot_estimates)
    stat = {}
    stat['estimator'] = 'T-learner'
    stat['estimate'] = full_estimates
    #stat['mean_boot'] = np.mean(est_boot)
    stat['SD'] = np.std(est_boot)
    stat['CI_radius'] = signif_level * stat['SD']
    stat['lower_ci'] = stat['estimate'] - stat['CI_radius']
    stat['upper_ci'] = stat['estimate'] + stat['CI_radius']
    est_stat.append(stat)

    return pd.DataFrame(est_stat)


def get_samples(data, control_treat_prop, n_training_set = 50000, n_testing_set = 2000):
    
    # randomly select 50000 training set and 2000 testing set
    # among them, we need to make w = 1 and w = 0 according to proportion, this is the permutation we have
    idx = list(np.arange(len(data)))
    random_idx = random.sample(idx, n_training_set + n_testing_set)
    # frist 50000 are training, last 2000 are testing
    train_sample = random_idx[:n_training_set]
    test_sample = random_idx[:n_testing_set]
    train_df = data.iloc[train_sample].copy()
    test_df = data.iloc[test_sample].copy()
    
    # assgin w = 1 according to proprtion
    w_train = np.zeros(len(train_sample))
    w_train[int(len(train_sample) * control_treat_prop) :] = 1 
    w_test = np.zeros(len(test_sample))
    w_test[int(len(test_sample) * control_treat_prop) :] = 1 
    
    # choose new Y
    y_train = []
    y_test = []
    for i in range(len(w_train)):
        if w_train[i] == 0:
            y_train.append(train_df.iloc[i].new_y0)
        else:
            y_train.append(train_df.iloc[i].new_y1)
    for i in range(len(w_test)):
        if w_test[i] == 0:
            y_test.append(test_df.iloc[i].new_y0)
        else:
            y_test.append(test_df.iloc[i].new_y1)
    # add new W,Y to both dfs respectively
    train_df = train_df.drop(columns = ['W','Y','new_y0','new_y1'])
    test_df = test_df.drop(columns = ['W','Y','new_y0','new_y1'])
    train_df['W'] = w_train
    train_df['Y'] = y_train
    test_df['W'] = w_test
    test_df['Y'] = y_test
    
    return train_df, test_df


def boostrap_exp(estimator,data,control_treat_prop, n_training_set = 50000, n_testing_set = 2000, full_ate = full_ate):
        bool_ates = []
        full_estimate_ate = full_ate
        # only get test_df 
        _, test_df = get_samples(data,control_treat_prop)
        for _ in range(10):
            t_learner = Tlearner(RandomForestClassifier(),RandomForestClassifier())
            train_df, _ = get_samples(data,control_treat_prop)
            # train T-learner
            t_learner.fit(train_df.drop(columns = ['W','Y']), train_df['W'], train_df['Y'])
            # calculate the ate for boot test set
            ite, _, _, _ = t_learner.get_ite(test_df.drop(columns = ['W','Y']), test_df['W'], test_df['Y'])
            bool_ates.append(np.mean(ite))
        
        return get_bootstrap_stats(bool_ates, full_estimate_ate)


In [20]:
control_treat_prop = len(d[d['W'] == 1]) / len(d)
result = boostrap_exp(T_learner, d, control_treat_prop )

In [21]:
result

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,0.08,0.006373,0.012491,0.067509,0.092491
